In [4]:
import pandas as pd
import os
path = '../datasets/ALPHAFOLD PDBs/'

In [9]:

pdb_files = []
for folder in os.listdir(path):
    for file in os.listdir(path + folder):
        if 'rank_001' in file and file.endswith('.pdb'):
            pdb_files.append(path + folder + '/' + file)



In [15]:
pdb_files

['../datasets/ALPHAFOLD PDBs/G185R_e6971/G185R_e6971_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/E168L_0f4af/E168L_0f4af_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/R53Q_8ee59/R53Q_8ee59_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/W97C_b9ccb/W97C_b9ccb_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/K353Q_ed31e/K353Q_ed31e_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/G360A_4d8ad/G360A_4d8ad_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/G360R_057f7/G360R_057f7_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/A267V_1c2c3/A267V_1c2c3_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/ALPHAFOLD PDBs/G251D_5ea06/G251D_5ea06_unrelaxed_rank_001_alphafold2_ptm_model_5_seed_000.pdb',
 '../datasets/

In [13]:
# for each pdb file in pdb_files, get name of the mutation which is the name of the folder before '_'
mutations = []
for pdb in pdb_files:
    mutations.append(pdb.split('/')[-2].split('_')[0])

    

In [14]:
mutations

['G185R',
 'E168L',
 'R53Q',
 'W97C',
 'K353Q',
 'G360A',
 'G360R',
 'A267V',
 'G251D',
 'L116P',
 'T167I',
 'F227S',
 'A122V',
 'Y40S',
 'G362E',
 'E168K',
 'S150L',
 'G361R',
 'G152R',
 'S47L',
 'L44F',
 'I216T',
 'E401Q',
 'E178G',
 'Y272C',
 'P332R',
 'G198D',
 'K248E',
 'R330S',
 'G161R',
 'G170S',
 'M368V',
 'P230S',
 'C120F',
 'I346T',
 'W97G',
 'D18Y',
 'V300G',
 'F73L',
 'G205D',
 'G170A',
 'R225P',
 'G270R',
 'G123A']

In [32]:
# create a dataframe with the mutation and the pdb file
df = pd.DataFrame({'mutation': mutations, 'pdb_file': pdb_files})
df.to_csv('../datasets/pdb_files.csv', index=False)
